<a href="https://colab.research.google.com/github/MaisieDeGraaf/Housing-Project_Final/blob/Pooja/linreg_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install pymongo

In [3]:
#Import our dependencies
import pandas as pd
from pymongo import MongoClient
from api_keys import mongo_username, mongo_password
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

ModuleNotFoundError: No module named 'api_keys'